In [1]:
import numpy as np
import pandas as pd

In [2]:
class MLP(object):
    model = None

    @staticmethod
    def f(net):
        return ( 1/ (1+ np.exp(-net)) )

    @staticmethod
    def df_dnet(f_net):
        return ( f_net * (1 - f_net) )

    def __init__(self, input_length=2, hidden_length=[3, 3], output_length=1, activation_function=f , d_activation_function=df_dnet):
        hidden_layer = list()
        previous_length = input_length
        for layer_length in hidden_length:
            hidden_layer.append(np.random.rand(layer_length, previous_length+1) - 0.5)
            previous_length = layer_length

        self.model = {
            'input_length': input_length, 
            'hidden_length': hidden_length, 
            'output_length': output_length, 
            'activation_function': activation_function.__func__, 
            'd_activation_function': d_activation_function.__func__,
            'hidden': hidden_layer,
            'output': (np.random.rand(output_length, previous_length+1) - 0.5),
        }

    def forward(self, x):
        # Recuperando valores do modelo
        hidden = self.model['hidden']
        hidden_length = len(self.model['hidden_length'])
        output = self.model['output']
        f = self.model['activation_function']
        df = self.model['d_activation_function']

        # Camadas Escondidas
        net_h = [None] * hidden_length
        f_net_h = [None] * hidden_length
        df_net_h = [None] * hidden_length
        
        
        # Adicionando 1 para multiplicar o Theta.
        previous_layer = np.pad(x, (0, 1), 'constant', constant_values=(1))
        for i in range(hidden_length):
            net_h[i] = np.sum(np.multiply(hidden[i], previous_layer), axis=1)
            f_net_h[i] = f(net_h[i])
            df_net_h[i] = df(f_net_h[i])
            previous_layer = np.pad(f_net_h[i], (0, 1), 'constant', constant_values=(1))

            
        # Camada de Saída
        net_o = np.sum(np.multiply(output, previous_layer), axis=1)
        f_net_o = f(net_o)
        df_net_o = df(f_net_o)

        # Retornando valores do forward.
        return {
            'net_h': net_h,
            'f_net_h': f_net_h,
            'df_net_h': df_net_h,
            'net_o': net_o,
            'f_net_o': f_net_o,
            'df_net_o': df_net_o,
        }

    def backpropagation(self, X, Y, eta=0.1, threshold=1e-3, alpha=0,max_inter = 200000):
        squaredError = 2*threshold
        hidden_length = len(self.model['hidden_length'])
        counter = 0
        
        dE2_dw_o_t = 0
        dE2_dw_h_t = [0] * hidden_length
        while(squaredError > threshold and counter < max_inter):
            squaredError = 0
            # Pra cada valor do conjunto de dados
            for x, y in zip(X, Y):
                # Calculando saída
                results = self.forward(x)
                #  Calculando o erro
                error = (y - results['f_net_o'])                
                squaredError += np.sum(np.power(error, 2))
                

                # Backwards camada de saída
                h = hidden_length - 1
                delta_o = error * results['df_net_o']
                f_net_h = np.pad(results['f_net_h'][h], (0, 1), 'constant', constant_values=(1))
                dE2_dw_o = np.multiply(np.array([-2*delta_o]).T, np.array([f_net_h]))
                
                # Backwards camada escondida
                delta_h = [None] * hidden_length
                dE2_dw_h = [None] * hidden_length
                delta = delta_o
                w_o_kj = self.model['output'][:,0:self.model['hidden_length'][h]] 

                # For reverso
                for i in reversed(range(1, hidden_length)):
                    # Cálculo das adaptações
                    delta_h[i] = np.array([results['df_net_h'][i]]) * np.dot(delta, w_o_kj)         
                    dE2_dw_h[i] = np.multiply(-2*delta_h[i].T,
                        np.pad(results['f_net_h'][i-1], (0, 1), 'constant', constant_values=(1)))
                    # Controlando as iterações
                    delta = delta_h[i]
                    w_o_kj = self.model['hidden'][i][:, 0:self.model['hidden_length'][i-1]]
                    
                delta_h[0] = np.array([results['df_net_h'][0]]) * np.dot(delta, w_o_kj)               
                dE2_dw_h[0] =  np.multiply(-2*delta_h[0].T,
                    np.pad(x, (0, 1), 'constant', constant_values=(1)))
                               
                
                # Aplicar adaptação na saída
                self.model['output'] = self.model['output'] - eta * dE2_dw_o - alpha*dE2_dw_o_t

                # Aplicar adaptação na escondida
                for i in range(hidden_length):
                    self.model['hidden'][i] = self.model['hidden'][i] - eta * dE2_dw_h[i] - alpha*dE2_dw_h_t[i]

                dE2_dw_o_t = dE2_dw_o
                dE2_dw_h_t = dE2_dw_h
                
            squaredError = squaredError / len(X) 
            counter += 1
            if(counter % 1000 == 0):
                print('error %.6lf - iter. %d' % (squaredError, counter))



In [3]:
class RBFN(object):
    def __init__(self, input_shape, hidden_shape, output_shape):
        self.input_shape = input_shape
        self.hidden_shape = hidden_shape
        self.sigmas = None
        self.centers = None
        self.mlp = MLP(input_length=self.hidden_shape,hidden_length=[3], output_length=output_shape)

    def _kernel_function(self, center, x, sigma):
        return np.exp(-(np.linalg.norm(center-x)**2)/(2*sigma**2) )
    
    
    def _kernel_layer_fit(self, X, centers, threshold=1e-5, sig_err=1e-5):
        ids = np.random.choice(X.shape[0], centers, replace=False)
        centroids = X[ids,]
        error = 2*threshold

        while (error > threshold):
            distances = []
            for c in centroids:
                distances.append(
                    np.apply_along_axis(lambda x, u: np.linalg.norm(x-u), 1, X, c)
                    )

            ids = np.argsort(np.array(distances).T)[:,0]
            error = 0
            for i in range(centers):
                old = centroids[i, ]
                centroids[i,] = np.mean(X[ids == i], axis=0)
                error = error + np.sqrt(np.sum(np.power(centroids[i,] - old, 2)))

        self.centers = centroids
        self.sigmas = [np.std(X[ids == i])+sig_err for i in range(centers)]
        self.centroid_to_x = ids


    def fit(self, X, Y, eta=0.1, alpha=0):
        self._kernel_layer_fit(X, self.hidden_shape)
        transformed_X = []
        for i in range(self.hidden_shape):
            transformed_X.append(np.apply_along_axis(self._kernel_function, 1, X, self.centers[i], self.sigmas[i]))

        transformed_X = np.array(transformed_X).T
        self.mlp.backpropagation(transformed_X, Y, eta=eta, alpha=alpha)

        
    def predict(self, x):
        transformed_x = []
        for i in range(self.hidden_shape):
            transformed_x.append(self._kernel_function(x, self.centers[i], self.sigmas[i]))
        return self.mlp.forward(transformed_x)['f_net_o'][0]


# xor_x = np.array([0, 0, 0, 1, 1, 0, 1, 1]).reshape((4,2))
# xor_y = np.array([x[0]^x[1] for x in xor_x])

# rbfn = RBFN(2, 3, 1)
# rbfn.fit(xor_x, xor_y, eta=0.8)

    
# for x, y in zip(xor_x, xor_y):
#     print(rbfn.predict(x))

### Preparando dataset

In [4]:
# Funcao p/ normalizar uma coluna de um dataframe
def norm_f(col):
    return (col - col.min())/(col.max() - col.min())

# Funcao p/ normalizar todas as colunas de um dataframe
def normalize_df(df):
    return df.apply(norm_f,axis=0)

# Fixing datset
with open('seeds_dataset.txt') as f:
    string = f.read().replace('\t\t','\t')
    with open('seeds_dataset_fixed.txt','w') as f2:
        f2.write(string)

# Loading dataset into dataframe
df = pd.read_csv('seeds_dataset_fixed.txt',header=None, sep='\t')
# Switching class for dummy variable
df = pd.concat([df.iloc[:,:7], pd.get_dummies(df[7],prefix='c')],axis=1)
# Normalizing dataframe to 0-1 interval
df = normalize_df(df)
# Shuffleing dataframe entries
df = df.sample(frac=1)

# Result
df.head(5)

,0,1,2,3,4,5,6,c_1,c_2,c_3
77,0.955619,0.995868,0.618875,0.945946,0.843906,0.479255,0.951256,0.0,1.0,0.0
169,0.061379,0.121901,0.252269,0.107545,0.060584,0.358333,0.280158,0.0,0.0,1.0
45,0.303116,0.336777,0.647005,0.268581,0.374198,0.103356,0.217627,1.0,0.0,0.0
59,0.143532,0.219008,0.282214,0.146396,0.286529,0.095815,0.000000,1.0,0.0,0.0
139,0.532578,0.572314,0.697822,0.547860,0.600143,0.390578,0.690793,0.0,1.0,0.0


In [5]:
# Splitting dataframe in data and target
data = df.iloc[:,:7]
target = df.iloc[:,7:]

### Treinando e testando MLP

In [6]:
# Treinando MLP
mlp = MLP(input_length = data.shape[1], hidden_length = [3] ,output_length = target.shape[1])
mlp.backpropagation(data.values.astype(np.float),target.values.astype(np.float))

error 0.024059 - iter. 1000
error 0.016340 - iter. 2000
error 0.013028 - iter. 3000
error 0.010889 - iter. 4000
error 0.008981 - iter. 5000
error 0.007309 - iter. 6000
error 0.006032 - iter. 7000
error 0.005097 - iter. 8000
error 0.004401 - iter. 9000
error 0.003864 - iter. 10000
error 0.003438 - iter. 11000
error 0.003091 - iter. 12000
error 0.002801 - iter. 13000
error 0.002556 - iter. 14000
error 0.002345 - iter. 15000
error 0.002163 - iter. 16000
error 0.002003 - iter. 17000
error 0.001862 - iter. 18000
error 0.001737 - iter. 19000
error 0.001625 - iter. 20000
error 0.001525 - iter. 21000
error 0.001435 - iter. 22000
error 0.001354 - iter. 23000
error 0.001280 - iter. 24000
error 0.001212 - iter. 25000
error 0.001151 - iter. 26000
error 0.001095 - iter. 27000
error 0.001043 - iter. 28000


In [ ]:
# Treinando RBFN
rbfn = RBFN(data.shape[1],3,target.shape[1])
rbfn.fit(data.values, target.values)

error 0.224226 - iter. 1000
error 0.210131 - iter. 2000
error 0.204859 - iter. 3000
error 0.201232 - iter. 4000
error 0.198621 - iter. 5000
error 0.197147 - iter. 6000
error 0.195824 - iter. 7000
error 0.194789 - iter. 8000
error 0.193979 - iter. 9000
error 0.193324 - iter. 10000
error 0.192768 - iter. 11000
error 0.192279 - iter. 12000
error 0.191836 - iter. 13000
error 0.191433 - iter. 14000
error 0.191065 - iter. 15000
error 0.190729 - iter. 16000
error 0.190423 - iter. 17000
error 0.190143 - iter. 18000
error 0.189883 - iter. 19000
error 0.189639 - iter. 20000
error 0.189407 - iter. 21000
error 0.189182 - iter. 22000
error 0.188957 - iter. 23000
error 0.188728 - iter. 24000
error 0.188488 - iter. 25000
error 0.188232 - iter. 26000
error 0.187956 - iter. 27000
error 0.187656 - iter. 28000
error 0.187332 - iter. 29000
error 0.186991 - iter. 30000
error 0.186641 - iter. 31000
error 0.186293 - iter. 32000
error 0.185957 - iter. 33000
error 0.185640 - iter. 34000
error 0.185346 - iter. 